In [1]:
# Imports and config
import sys
import os

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root)

import torch
import json

from src.data_loader import get_data_loaders
from src.models.squeezenet import SqueezeNet
from src.evaluate import evaluate_model
from src.utils import save_results

# Config
DATA_DIR = "../data/FER2013"
BATCH_SIZE = 64
DEVICE = 'cpu'

# Load data
train_loader, test_loader = get_data_loaders(DATA_DIR, batch_size=BATCH_SIZE)
print("Device:", DEVICE)

Device: cpu


In [2]:
# Helper to load saved model and evaluate
results = {}

def load_and_eval(model_cls, model_path, model_name):
    model = model_cls(num_classes=7, in_channels=1)

    if os.path.exists(model_path):
        state = torch.load(model_path, map_location=DEVICE, weights_only=True)
        model.load_state_dict(state)
        print("\n" + "="*50)
        print(f"Loading and evaluating {model_name} model...")
        print("="*50)
        
        r = evaluate_model(model, test_loader, DEVICE)
        results[model_name] = r
        
        print(f"\nAccuracy: {r["accuracy"]:.2f}%")
        print(f"Inference Time: {r["inference_time_sec"]:.2f}s")
        print(f"Energy Consumed: {r["energy_consumed_kwh"]:.6f} kWh")
        print(f"CO₂ Emissions: {r["co2_emissions_kg"]:.6f} kg")
        print(f"FLOPs: {r["flops"]}")
        print(f"Parameters: {r["parameters"]}\n")
        
        # Save results
        save_results(results, path="../results.json")
    else:
        print(f"{model_name} checkpoint not found:", model_path)
    return 

In [4]:
# Evaluate SqueezeNet
squeezenet_path = "../models/squeezenet/best_model.pth"

load_and_eval(SqueezeNet, squeezenet_path, "SqueezeNet")


Loading and evaluating SqueezeNet model...

Accuracy: 62.62%
Inference Time: 16.88s
Energy Consumed: 0.000753 kWh
CO₂ Emissions: 0.000478 kg
FLOPs: 32.01 MMac
Parameters: 735.56 k

Results saved to ../results.json
